# Assignment Module 2: Product Classification

The goal of this assignment is to implement a neural network that classifies smartphone pictures of products found in grocery stores. The assignment will be divided into two parts: first, you will be asked to implement from scratch your own neural network for image classification; then, you will fine-tune a pretrained network provided by PyTorch.


# Assignment Module 2: Product Classification

The goal of this assignment is to implement a neural network that classifies smartphone pictures of products found in grocery stores. The assignment will be divided into two parts: first, you will be asked to implement from scratch your own neural network for image classification; then, you will fine-tune a pretrained network provided by PyTorch.


## Preliminaries: the dataset

The dataset you will be using contains natural images of products taken with a smartphone camera in different grocery stores:

<p align="center">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Granny-Smith.jpg" width="150">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Pink-Lady.jpg" width="150">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Lemon.jpg" width="150">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Banana.jpg" width="150">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Vine-Tomato.jpg" width="150">
</p>
<p align="center">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Yellow-Onion.jpg" width="150">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Green-Bell-Pepper.jpg" width="150">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Arla-Standard-Milk.jpg" width="150">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Oatly-Natural-Oatghurt.jpg" width="150">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Alpro-Fresh-Soy-Milk.jpg" width="150">
</p>

The products belong to the following 43 classes:
```
0.  Apple
1.  Avocado
2.  Banana
3.  Kiwi
4.  Lemon
5.  Lime
6.  Mango
7.  Melon
8.  Nectarine
9.  Orange
10. Papaya
11. Passion-Fruit
12. Peach
13. Pear
14. Pineapple
15. Plum
16. Pomegranate
17. Red-Grapefruit
18. Satsumas
19. Juice
20. Milk
21. Oatghurt
22. Oat-Milk
23. Sour-Cream
24. Sour-Milk
25. Soyghurt
26. Soy-Milk
27. Yoghurt
28. Asparagus
29. Aubergine
30. Cabbage
31. Carrots
32. Cucumber
33. Garlic
34. Ginger
35. Leek
36. Mushroom
37. Onion
38. Pepper
39. Potato
40. Red-Beet
41. Tomato
42. Zucchini
```

The dataset is split into training (`train`), validation (`val`), and test (`test`) set.

The following code cells download the dataset and define a `torch.utils.data.Dataset` class to access it. This `Dataset` class will be the starting point of your assignment: use it in your own code and build everything else around it.

In [1]:

!git clone https://github.com/marcusklasson/GroceryStoreDataset.git 

Cloning into 'GroceryStoreDataset'...
Updating files:  21% (1218/5717)
Updating files:  22% (1258/5717)
Updating files:  23% (1315/5717)
Updating files:  24% (1373/5717)
Updating files:  25% (1430/5717)
Updating files:  26% (1487/5717)
Updating files:  27% (1544/5717)
Updating files:  28% (1601/5717)
Updating files:  29% (1658/5717)
Updating files:  30% (1716/5717)
Updating files:  31% (1773/5717)
Updating files:  32% (1830/5717)
Updating files:  33% (1887/5717)
Updating files:  34% (1944/5717)
Updating files:  35% (2001/5717)
Updating files:  36% (2059/5717)
Updating files:  37% (2116/5717)
Updating files:  37% (2123/5717)
Updating files:  38% (2173/5717)
Updating files:  39% (2230/5717)
Updating files:  40% (2287/5717)
Updating files:  41% (2344/5717)
Updating files:  42% (2402/5717)
Updating files:  43% (2459/5717)
Updating files:  44% (2516/5717)
Updating files:  45% (2573/5717)
Updating files:  46% (2630/5717)
Updating files:  47% (2687/5717)
Updating files:  48% (2745/5717)
Updat

This command is used to create GroceryStoreDataset named directory in the current working directory
It contains all dataset files 

In [2]:
from pathlib import Path # class for handling file paths 
from PIL import Image # class for opening and manipulating images from the Python imaging library 
from typing import List, Tuple # typing library provides about typing hints for code readability and error checking

import torch 
from torch import Tensor
from torch.utils.data import DataLoader, Dataset
from torch.optim import Adam, SGD, AdamW
import torch.nn.functional as F
import torch.nn as nn
import torchvision
# Torch and TorchVision libraries are significant libraries for building and training neural networks.

from tqdm.notebook import tqdm
# tqdm is a library that provides a progress bar for loops and other iterable objects. It is useful
import matplotlib.pyplot as plt
# matplotlib is a library for creating static, animated, and interactive visualizations in Python.
import numpy as np
# used for numerical operations 


import random


# Check device
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# Check if GPU is available and use it if it is. Otherwise, use the CPU.
print(f"Device: {device}")

# Fix random seed
def fix_random(seed: int) -> None:  
    np.random.seed(seed)
    random.seed(seed)
    # 
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

fix_random(45)

# Define the GroceryStoreDataset class
class GroceryStoreDataset(Dataset):
    def __init__(self, split: str, transform=None) -> None:
        super().__init__()
        self.root = Path("GroceryStoreDataset/dataset")
        self.split = split
        self.paths, self.labels = self.read_file()
        self.transform = transform

    def __len__(self) -> int:
        return len(self.labels)

    def __getitem__(self, idx) -> Tuple[torch.Tensor, int]:
        img = Image.open(self.root / self.paths[idx])
        label = self.labels[idx]
        if self.transform:
            img = self.transform(img)
        return img, label

    def read_file(self) -> Tuple[List[str], List[int]]:
        paths, labels = [], []
        with open(self.root / f"{self.split}.txt") as f:
            for line in f:
                path, _, label = line.strip().split(", ")
                paths.append(path)
                labels.append(int(label))
        return paths, labels

    def get_num_classes(self) -> int:
        return max(self.labels) + 1



Device: cpu


In [3]:
# Initial transform
transform = torchvision.transforms.Compose([
    torchvision.transforms.ToTensor()
])

In [4]:
# Load datasets
train_dataset = GroceryStoreDataset(split='train', transform=transform)
val_dataset = GroceryStoreDataset(split='val', transform=transform)
test_dataset = GroceryStoreDataset(split='test', transform=transform)


In [5]:
# Checking image shapes
def check_image_shapes(dataset):
    w, h = 0, 0
    for i in range(len(dataset)):
        img, _ = dataset[i]
        if img.shape[1] > w:
            w = img.shape[1]
        if img.shape[2] > h:
            h = img.shape[2]
    return w, h

w_t, h_t = check_image_shapes(train_dataset)
print(w_t, h_t)  # Example output: 464, 464

464 464


In [6]:
from torchvision import transforms

# Define the transform to resize images
image_height, image_width, image_channels = 348, 348, 3
resize_transform = transforms.Compose([
    transforms.Resize((image_height, image_width)),
    transforms.ToTensor()
])

In [7]:
# Load the training dataset with the resizing transform
train_dataset = GroceryStoreDataset(split='train', transform=resize_transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=False)

In [8]:
# Compute mean and standard deviation
mean = torch.zeros(3)
std = torch.zeros(3)
nb_samples = 0

for images, _ in train_loader:
    batch_samples = images.size(0)
    images = images.view(batch_samples, images.size(1), -1)
    mean += images.mean(2).sum(0)
    std += images.std(2).sum(0)
    nb_samples += batch_samples

mean /= nb_samples
std /= nb_samples

print(f'Mean: {mean}')
print(f'Std: {std}')

Mean: tensor([0.5306, 0.3964, 0.2564])
Std: tensor([0.2325, 0.2093, 0.1781])


In [9]:
# Define accuracy and training loop functions
def ncorrect(scores, y):
    y_hat = torch.argmax(scores, -1)
    return (y_hat == y).sum()

def accuracy(scores, y):
    correct = ncorrect(scores, y)
    return correct.true_divide(y.shape[0])

def train_loop(model, train_dl, epochs, opt, val_dl=None, verbose=False, label_smoothing=0):
    best_val_acc = 0
    best_params = []
    best_epoch = -1
    for e in tqdm(range(epochs)):
        model.train()
        train_loss, train_samples, train_acc = 0, 0, 0
        for train_data in train_dl:
            imgs = train_data[0].to(device)
            labels = train_data[1].to(device)
            scores = model(imgs)
            loss = F.cross_entropy(scores, labels, label_smoothing=label_smoothing)
            train_loss += loss.item() * imgs.shape[0]
            train_samples += imgs.shape[0]
            train_acc += ncorrect(scores, labels).item()
            opt.zero_grad()
            loss.backward()
            opt.step()
        train_acc /= train_samples
        train_loss /= train_samples

        model.eval()
        with torch.no_grad():
            val_loss, val_samples, val_acc = 0, 0, 0
            if val_dl is not None:
                for val_data in val_dl:
                    imgs = val_data[0].to(device)
                    labels = val_data[1].to(device)
                    val_scores = model(imgs)
                    val_loss += F.cross_entropy(val_scores, labels).item() * imgs.shape[0]
                    val_samples += imgs.shape[0]
                    val_acc += ncorrect(val_scores, labels).item()
                val_acc /= val_samples
                val_loss /= val_samples
                if val_acc > best_val_acc:
                    best_val_acc = val_acc
                    best_params = model.state_dict()
                    torch.save(best_params, "best_model.pth")
                    best_epoch = e

        if verbose:
            print(f"Epoch {e}: train loss {train_loss:.3f} - train acc {train_acc:.3f}" + ("" if val_dl is None else f" - valid loss {val_loss:.3f} - valid acc {val_acc:.3f}"))
            if val_acc >= 0.60:
                best_params = model.state_dict()
                torch.save(best_params, "A2.pth")
                best_epoch = e
                break

    if verbose and val_dl is not None:
        print(f"Best epoch {best_epoch}, best acc {best_val_acc}")

    return best_val_acc, best_params, best_epoch

In [10]:
# Define the CNN
class ExtendedCNN(nn.Module):
    def __init__(self, channels=16, num_classes=10, p_dropout=0):
        super(ExtendedCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=image_channels, out_channels=channels, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=channels, out_channels=channels*2, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=channels*2, out_channels=channels*4, kernel_size=3, stride=1, padding=1)
        self.conv4 = nn.Conv2d(in_channels=channels*4, out_channels=channels*8, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        def conv2d_size_out(size, kernel_size=3, stride=1, padding=1):
            return (size - kernel_size + 2 * padding) // stride + 1
        def conv2d_compute_size(conv_number, kernels, max_poolings):
            image_w = image_width
            for i in range(conv_number):
                image_w = conv2d_size_out(image_w, kernel_size=kernels[i])
                if max_poolings[i] == 1:
                    image_w = image_w // 2
            return image_w
        image_w = conv2d_compute_size(4, [3, 3, 3, 3], [1, 1, 1, 1])
        linear_input_size = image_w * image_w * channels * 8
        self.fc1 = nn.Linear(linear_input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.pool(F.relu(self.conv4(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [11]:
# Hyperparameters and DataLoader
transform_2 = torchvision.transforms.Compose([
    torchvision.transforms.RandomResizedCrop(size=(image_height, image_width), antialias=True),
    torchvision.transforms.RandomHorizontalFlip(p=0.5),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean, std)
])

val_transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize((image_height, image_width)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean, std)
])

train_dataset = GroceryStoreDataset(split='train', transform=transform_2)
val_dataset = GroceryStoreDataset(split='val', transform=val_transforms)

batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)


In [13]:
# Model training
new_model = ExtendedCNN(num_classes=train_dataset.get_num_classes())
new_model.to(device)

optimizer = Adam(new_model.parameters(), lr=0.001)
epochs = 80
label_smoothing = 0.1

best_val_acc, best_params, best_epoch = train_loop(
    new_model,
    train_loader,
    epochs,
    optimizer,
    val_loader,
    verbose=True,
    label_smoothing=label_smoothing
)


  0%|          | 0/80 [00:00<?, ?it/s]

Epoch 0: train loss 3.160 - train acc 0.186 - valid loss 2.925 - valid acc 0.220
Epoch 1: train loss 2.706 - train acc 0.293 - valid loss 2.571 - valid acc 0.304
Epoch 2: train loss 2.379 - train acc 0.392 - valid loss 2.309 - valid acc 0.341
Epoch 3: train loss 2.191 - train acc 0.441 - valid loss 2.170 - valid acc 0.375
Epoch 4: train loss 2.025 - train acc 0.503 - valid loss 2.079 - valid acc 0.365
Epoch 5: train loss 1.930 - train acc 0.541 - valid loss 2.205 - valid acc 0.372
Epoch 6: train loss 1.816 - train acc 0.578 - valid loss 1.974 - valid acc 0.395
Epoch 7: train loss 1.776 - train acc 0.591 - valid loss 2.025 - valid acc 0.368
Epoch 8: train loss 1.662 - train acc 0.650 - valid loss 2.147 - valid acc 0.341
Epoch 9: train loss 1.667 - train acc 0.646 - valid loss 1.943 - valid acc 0.426
Epoch 10: train loss 1.611 - train acc 0.661 - valid loss 2.204 - valid acc 0.382
Epoch 11: train loss 1.551 - train acc 0.690 - valid loss 2.273 - valid acc 0.436
Epoch 12: train loss 1.514

In [ ]:
# Grid Search
weight_decays = [0.0, 0.01, 0.001]
label_smoothings = [0.0, 0.05, 0.1, 0.15, 0.2]
lrs = [0.001]
channels = [16]
batches = [8]

epochs = 80
test_iteration = 1
total_tests = len(weight_decays) * len(label_smoothings) * len(lrs) * len(channels) * len(batches)

for weight_decay in weight_decays:
    for label_smoothing in label_smoothings:
        for lr in lrs:
            for channel in channels:
                for batch in batches:
                    model = ExtendedCNN(channels=channel, num_classes=train_dataset.get_num_classes())
                    model.to(device)
                    optimizer = Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
                    print(f"Training with: weight_decay={weight_decay}, label_smoothing={label_smoothing}, lr={lr}, channels={channel}, batch_size={batch}, test={test_iteration}/{total_tests}")
                    test_iteration += 1
                    best_val_acc, best_params, best_epoch = train_loop(
                        model,
                        DataLoader(train_dataset, batch_size=batch, shuffle=True),
                        epochs,
                        optimizer,
                        DataLoader(val_dataset, batch_size=batch, shuffle=False),
                        verbose=True,
                        label_smoothing=label_smoothing
                    )
                    print(f"Finished training with best_val_acc={best_val_acc:.3f} at epoch {best_epoch}")


In [ ]:
# Testing on the test dataset
test_dataset = GroceryStoreDataset(split='test', transform=val_transforms)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
# Load the best model
best_model = ExtendedCNN(num_classes=train_dataset.get_num_classes())
best_model.load_state_dict(torch.load("best_model.pth"))
best_model.to(device)
best_model.eval()


In [ ]:
# Evaluate the model
test_acc = 0
test_samples = 0

with torch.no_grad():
    for test_data in test_loader:
        imgs = test_data[0].to(device)
        labels = test_data[1].to(device)
        scores = best_model(imgs)
        test_acc += ncorrect(scores, labels).item()
        test_samples += imgs.size(0)

test_acc /= test_samples
print(f"Test accuracy: {test_acc:.3f}")

# Example usage of the best model
def predict(image_path):
    image = Image.open(image_path)
    image = val_transforms(image).unsqueeze(0).to(device)
    best_model.eval()
    with torch.no_grad():
        scores = best_model(image)
        predicted_label = torch.argmax(scores, -1).item()
    return predicted_label

In [ ]:
# Predict an example
example_image_path = "GroceryStoreDataset/dataset/example.jpg"  # Replace with an actual image path from your dataset
predicted_label = predict(example_image_path)
print(f"Predicted label for the example image: {predicted_label}")


## Part 1: design your own network

Your goal is to implement a convolutional neural network for image classification and train it on `GroceryStoreDataset`. You should consider yourselves satisfied once you obtain a classification accuracy on the **validation** split of **around 60%**. You are free to achieve that however you want, except for a few rules you must follow:

- You **cannot** simply instantiate an off-the-self PyTorch network. Instead, you must construct your network as a composition of existing PyTorch layers. In more concrete terms, you can use e.g. `torch.nn.Linear`, but you **cannot** use e.g. `torchvision.models.alexnet`.

- Justify every *design choice* you make. Design choices include network architecture, training hyperparameters, and, possibly, dataset preprocessing steps. You can either (i) start from the simplest convolutional network you can think of and add complexity one step at a time, while showing how each step gets you closer to the target ~60%, or (ii) start from a model that is already able to achieve the desired accuracy and show how, by removing some of its components, its performance drops (i.e. an *ablation study*). You can *show* your results/improvements however you want: training plots, console-printed values or tables, or whatever else your heart desires: the clearer, the better.

Don't be too concerned with your network performance: the ~60% is just to give you an idea of when to stop. Keep in mind that a thoroughly justified model with lower accuracy will be rewarded **more** points than a poorly experimentally validated model with higher accuracy.

## Part 2: fine-tune an existing network

Your goal is to fine-tune a pretrained **ResNet-18** model on `GroceryStoreDataset`. Use the implementation provided by PyTorch, do not implement it yourselves! (i.e. exactly what you **could not** do in the first part of the assignment). Specifically, you must use the PyTorch ResNet-18 model pretrained on ImageNet-1K (V1). Divide your fine-tuning into two parts:

1. First, fine-tune the Resnet-18 with the same training hyperparameters you used for your best model in the first part of the assignment.
1. Then, tweak the training hyperparameters in order to increase the accuracy on the validation split of `GroceryStoreDataset`. Justify your choices by analyzing the training plots and/or citing sources that guided you in your decisions (papers, blog posts, YouTube videos, or whatever else you find enlightening). You should consider yourselves satisfied once you obtain a classification accuracy on the **validation** split **between 80 and 90%**.

## Preliminaries: the dataset

The dataset you will be using contains natural images of products taken with a smartphone camera in different grocery stores:

<p align="center">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Granny-Smith.jpg" width="150">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Pink-Lady.jpg" width="150">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Lemon.jpg" width="150">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Banana.jpg" width="150">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Vine-Tomato.jpg" width="150">
</p>
<p align="center">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Yellow-Onion.jpg" width="150">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Green-Bell-Pepper.jpg" width="150">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Arla-Standard-Milk.jpg" width="150">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Oatly-Natural-Oatghurt.jpg" width="150">
  <img src="https://github.com/marcusklasson/GroceryStoreDataset/raw/master/sample_images/natural/Alpro-Fresh-Soy-Milk.jpg" width="150">
</p>

The products belong to the following 43 classes:
```
0.  Apple
1.  Avocado
2.  Banana
3.  Kiwi
4.  Lemon
5.  Lime
6.  Mango
7.  Melon
8.  Nectarine
9.  Orange
10. Papaya
11. Passion-Fruit
12. Peach
13. Pear
14. Pineapple
15. Plum
16. Pomegranate
17. Red-Grapefruit
18. Satsumas
19. Juice
20. Milk
21. Oatghurt
22. Oat-Milk
23. Sour-Cream
24. Sour-Milk
25. Soyghurt
26. Soy-Milk
27. Yoghurt
28. Asparagus
29. Aubergine
30. Cabbage
31. Carrots
32. Cucumber
33. Garlic
34. Ginger
35. Leek
36. Mushroom
37. Onion
38. Pepper
39. Potato
40. Red-Beet
41. Tomato
42. Zucchini
```

The dataset is split into training (`train`), validation (`val`), and test (`test`) set.

The following code cells download the dataset and define a `torch.utils.data.Dataset` class to access it. This `Dataset` class will be the starting point of your assignment: use it in your own code and build everything else around it.

## Part 1: design your own network

Your goal is to implement a convolutional neural network for image classification and train it on `GroceryStoreDataset`. You should consider yourselves satisfied once you obtain a classification accuracy on the **validation** split of **around 60%**. You are free to achieve that however you want, except for a few rules you must follow:

- You **cannot** simply instantiate an off-the-self PyTorch network. Instead, you must construct your network as a composition of existing PyTorch layers. In more concrete terms, you can use e.g. `torch.nn.Linear`, but you **cannot** use e.g. `torchvision.models.alexnet`.

- Justify every *design choice* you make. Design choices include network architecture, training hyperparameters, and, possibly, dataset preprocessing steps. You can either (i) start from the simplest convolutional network you can think of and add complexity one step at a time, while showing how each step gets you closer to the target ~60%, or (ii) start from a model that is already able to achieve the desired accuracy and show how, by removing some of its components, its performance drops (i.e. an *ablation study*). You can *show* your results/improvements however you want: training plots, console-printed values or tables, or whatever else your heart desires: the clearer, the better.

Don't be too concerned with your network performance: the ~60% is just to give you an idea of when to stop. Keep in mind that a thoroughly justified model with lower accuracy will be rewarded **more** points than a poorly experimentally validated model with higher accuracy.

## Part 2: fine-tune an existing network

Your goal is to fine-tune a pretrained **ResNet-18** model on `GroceryStoreDataset`. Use the implementation provided by PyTorch, do not implement it yourselves! (i.e. exactly what you **could not** do in the first part of the assignment). Specifically, you must use the PyTorch ResNet-18 model pretrained on ImageNet-1K (V1). Divide your fine-tuning into two parts:

1. First, fine-tune the Resnet-18 with the same training hyperparameters you used for your best model in the first part of the assignment.
1. Then, tweak the training hyperparameters in order to increase the accuracy on the validation split of `GroceryStoreDataset`. Justify your choices by analyzing the training plots and/or citing sources that guided you in your decisions (papers, blog posts, YouTube videos, or whatever else you find enlightening). You should consider yourselves satisfied once you obtain a classification accuracy on the **validation** split **between 80 and 90%**.